In [30]:
import pandas as pd

# File paths
data_path = r"C:/Users/DELL/Desktop/Reconzo_Product_Analyst_Assignment/data/"
output_path = r"C:/Users/DELL/Desktop/Reconzo_Product_Analyst_Assignment/outputs/"
oms_file = data_path + "oms.csv"
pf_file = data_path + "pf.csv"
output_file = output_path + "merged_data.csv"

# Load datasets
oms_df = pd.read_csv(oms_file)
pf_df = pd.read_csv(pf_file)

# Normalize column names
oms_df.columns = oms_df.columns.str.lower()
pf_df.columns = pf_df.columns.str.lower()

# Convert 'sku' to string type
oms_df['sku'] = oms_df['sku'].astype(str)
pf_df['sku'] = pf_df['sku'].astype(str)

# Ensure key columns exist
required_columns = {'sku', 'awbno', 'getstatus'}
if not required_columns.issubset(oms_df.columns):
    raise ValueError("oms.csv is missing required columns")
if not {'sku', 'awbno'}.issubset(pf_df.columns):
    raise ValueError("pf.csv is missing required columns")

# Merge datasets on 'sku' and 'awbno'
merged_df = pd.merge(oms_df, pf_df, on=['sku', 'awbno'], how='outer', indicator=True)

# Define final inventory status logic
def determine_status(row):
    if row['_merge'] == 'both':
        return row['getstatus']  # Match found
    elif row['_merge'] == 'left_only':
        return "missing"  # awbno not found in pf.csv
    elif row['_merge'] == 'right_only':
        return "missing"  # awbno not found in oms.csv
    else:
        return row['getstatus'] + "-sku-mismatch" if pd.notna(row['getstatus']) else "sku-mismatch"

# Apply logic to create 'final inventory status'
merged_df['final inventory status'] = merged_df.apply(determine_status, axis=1)

# Drop unnecessary columns
merged_df.drop(columns=['_merge'], inplace=True)

# Save merged data
merged_df.to_csv(output_file, index=False)

print(f"Merged data saved to {output_file}")


Merged data saved to C:/Users/DELL/Desktop/Reconzo_Product_Analyst_Assignment/outputs/merged_data.csv
